# Top-1 Momentum Backtest (NautilusTrader) with Best-Index Selection

This notebook:
- Optionally evaluates multiple indices and picks the best index for the main backtest.
- Downloads Adjusted Close prices and builds daily bars for the selected index.
- Runs a Top-1 momentum strategy (ROC × slope of log-price), switch-only (all-in/all-out), bi-weekly rebalances, liquidation on stop.
- Prints engine-native PnL stats and a trade log.
- Shows last 5-day momentum score tables (raw and annualized), transposed.


In [1]:
# If running in a fresh environment, uncomment to install:
# %pip install yfinance pandas beautifulsoup4 lxml html5lib nautilus-trader

# ------------------------------
# Parameters (edit these)
# ------------------------------
START_DATE = '2022-01-01'        # backtest start for main engine
END_DATE   = '2025-08-21'        # backtest end for main engine
LOOKBACK   = 250                 # momentum slope window
ROC        = 250                 # ROC period in days
NUM_STOCKS = 1                   # top-N to hold (this notebook uses top-1 switch-only)
REB_DAYS   = 14                  # rebalance cadence in calendar days
POS_SIZE   = 0.95                # % of equity to deploy
TX_COST    = 7.0                 # fixed per-trade (engine may account in its own way)

# Optional: separate time frame to evaluate indices (NO per-index engine backtests).
# If both dates are non-empty, indices are ranked by TOP percent increase among their constituents
# over this window. If left empty, indices are ranked by a momentum proxy (last-day ROC × slope top-1).
INDEX_CHECK_START = '2022-01-01'            # e.g., '2023-01-01' or '' to disable
INDEX_CHECK_END   = '2025-08-21'            # e.g., '2023-12-31' or '' to disable

# Best index selection (applies either percent-increase or momentum-proxy, see above).
SELECT_BEST_INDEX = True
CANDIDATE_INDICES = ['SP500','SP500_IT','NASDAQ100','DAX40','FTSE100','CAC40','NIFTY50','KOSPI200']


In [2]:
# ------------------------------
# Imports (engine + helpers)
# ------------------------------
import os, sys
import pandas as pd
import numpy as np
import yfinance as yf

# Make the local strategy module importable
sys.path.append(os.path.abspath('stock_enhanced'))
from nautilus_engine_momentum import MomentumConfig, MomentumStrategy
from nautilus_trader.backtest.engine import BacktestEngine, BacktestEngineConfig
from nautilus_trader.model.identifiers import InstrumentId, Symbol
from nautilus_trader.model.venues import Venue
from nautilus_trader.model.instruments.equity import Equity
from nautilus_trader.model.objects import Currency, Price, Quantity, Money
from nautilus_trader.model.enums import CurrencyType, OmsType, AccountType, BookType
from nautilus_trader.model.data import Bar, BarType

# Index fetchers (Wikipedia scraping)
import importlib
idx = importlib.import_module('index_fetchers')

# ---------------------------------------------
# Utilities: download adjusted closes + scoring
# ---------------------------------------------
def get_adjusted_close(tickers):
    """Download Adjusted Close, clean, and return a wide DataFrame.
    Columns are tickers, index is DatetimeIndex.
    """
    data = yf.download(tickers, start=START_DATE, end=END_DATE, progress=False, group_by='column')
    if isinstance(data.columns, pd.MultiIndex) and 'Adj Close' in data.columns.get_level_values(0):
        close = data['Adj Close']
    else:
        close = data['Close'] if isinstance(data.columns, pd.MultiIndex) else data
    return close.ffill().bfill().dropna(axis=1, how='any')

def last_day_momentum_proxy(close_df, lookback, roc_period):
    """Compute last-day momentum proxy (ROC × slope) for an index universe.
    Returns the average of the top 3 scores among constituents (or fewer if <3 available).
    Returns the best (max) score among constituents.
    """
    i = len(close_df.index) - 1
    if i < max(lookback, roc_period):
        return None
    scores = []
    for col in close_df.columns:
        try:
            cur  = float(close_df.iloc[i, close_df.columns.get_loc(col)])
            past = float(close_df.iloc[i - roc_period, close_df.columns.get_loc(col)])
            if np.isnan(cur) or np.isnan(past) or past <= 0:
                continue
            roc = (cur - past) / past * 100.0
            roc = max(0.0, min(1.0, float(np.ceil(roc))))
            series = pd.Series(close_df[col].iloc[i - lookback + 1:i + 1]).replace(0, np.nan).dropna()
            if len(series) < 20:
                continue
            x = np.arange(len(series))
            slope = np.polyfit(x, np.log(series.values), 1)[0]
            score = roc * slope
            if np.isfinite(score):
                scores.append(float(score))
        except Exception:
            continue
    if not scores:
        return None
    scores.sort(reverse=True)
    topk = scores[:min(3, len(scores))]
    return float(np.mean(topk))

def best_index_by_percent_increase(close_df, start, end):
    """Compute TOP percent increase among constituents over [start, end].
    Returns that top percent increase (float) or None if insufficient.
    """
    cwin = close_df.loc[start:end].dropna(axis=1, how='any')
    if cwin.shape[1] > 0 and cwin.shape[0] > 1:
        pct_increase = (cwin.iloc[-1] / cwin.iloc[0] - 1.0) * 100.0
        return float(pct_increase.max()) if not pct_increase.empty else None
    return None

def choose_best_index(candidate_indices):
    """Rank indices either by percent increase window (if set) or momentum proxy.
    Returns (results_df, best_code, best_close).
    """
    results = []
    best_code = None
    best_score = None
    best_close = None
    for code in candidate_indices:
        try:
            tks = idx.get_index_constituents(code)
            c = get_adjusted_close(tks)
            if c.shape[1] == 0:
                continue
            if INDEX_CHECK_START and INDEX_CHECK_END:
                score = best_index_by_percent_increase(c, INDEX_CHECK_START, INDEX_CHECK_END)
            else:
                score = last_day_momentum_proxy(c, LOOKBACK, ROC)
            results.append({'index': code, 'score': score, 'num_symbols': c.shape[1]})
            if score is not None and (best_score is None or score > best_score):
                best_score = score
                best_code = code
                best_close = c
        except Exception as e:
            results.append({'index': code, 'score': None, 'num_symbols': 0, 'error': str(e)})
            continue
    results_df = pd.DataFrame(results).sort_values(by='score', ascending=False)
    return results_df, best_code, best_close


In [3]:
# ---------------------------------------------
# Pick best index (if enabled) and obtain `close`
# ---------------------------------------------
if SELECT_BEST_INDEX:
    results_df, best_code, best_close = choose_best_index(CANDIDATE_INDICES)
    print('Index candidates sorted by score (highest first):')
    display(results_df)
    if best_close is not None:
        print(f'Selected best index: {best_code} (score={results_df.iloc[0]["score"]})')
        close = best_close
    else:
        print('No best index selected, defaulting to NASDAQ-100.')
        close = get_adjusted_close(idx.get_nasdaq100())
else:
    close = get_adjusted_close(idx.get_nasdaq100())
print(f'Using {len(close.columns)} tickers')
close.tail(2)


/tmp/ipykernel_241254/1730580707.py:31: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=START_DATE, end=END_DATE, progress=False, group_by='column')
/tmp/ipykernel_241254/1730580707.py:31: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=START_DATE, end=END_DATE, progress=False, group_by='column')
/tmp/ipykernel_241254/1730580707.py:31: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=START_DATE, end=END_DATE, progress=False, group_by='column')
/tmp/ipykernel_241254/1730580707.py:31: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=START_DATE, end=END_DATE, progress=False, group_by='column')
/tmp/ipykernel_241254/1730580707.py:31: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(t

Index candidates sorted by score (highest first):


,index,score,num_symbols
7,KOSPI200,2135.639939,200
3,DAX40,1830.563884,41
1,SP500_IT,834.020221,68
0,SP500,834.020221,503
2,NASDAQ100,741.931941,101
4,FTSE100,707.465727,100
6,NIFTY50,458.237815,50
5,CAC40,227.323025,40


Selected best index: KOSPI200 (score=2135.639938828147)
Using 200 tickers


Ticker,000080.KS,000100.KS,000120.KS,000150.KS,000210.KS,000240.KS,000270.KS,000660.KS,000670.KS,000720.KS,...,375500.KS,377300.KS,383220.KS,402340.KS,443060.KS,450080.KS,454910.KS,456040.KS,457190.KS,489790.KS
Date,,,,,,,,,,,,,,,,,,,,,
2025-08-19,19120.0,113800.0,82200.0,525000.0,38000.0,21204.804688,104000.0,263000.0,39050.0,61500.0,...,42350.0,67500.0,61800.0,140400.0,186900.0,53300.0,60300.0,57800.0,46100.0,55300.0
2025-08-20,19230.0,113300.0,83100.0,508000.0,37200.0,21550.000000,105100.0,255500.0,38500.0,58800.0,...,41500.0,64300.0,63600.0,137100.0,186900.0,52400.0,59000.0,57200.0,45850.0,53100.0


In [4]:
# ---------------------------------------------
# Build instruments, bars, and run Nautilus engine
# ---------------------------------------------
def to_instrument_id(t: str) -> InstrumentId:
    return InstrumentId(Symbol(t), Venue('SIM'))

def to_equity(iid: InstrumentId) -> Equity:
    cur = Currency('USD', 2, 840, 'US Dollar', CurrencyType.FIAT)
    px_inc = Price(0.01, 2)
    lot = Quantity(1, 0)
    import time
    ts = int(time.time_ns())
    return Equity(iid, iid.symbol, cur, 2, px_inc, lot, ts, ts)

def bars_from_series(s: pd.Series, iid: InstrumentId):
    s = s.dropna()
    last_bt = BarType.from_str(f'{iid.value}-1-DAY-LAST-EXTERNAL')
    bid_bt  = BarType.from_str(f'{iid.value}-1-DAY-BID-EXTERNAL')
    ask_bt  = BarType.from_str(f'{iid.value}-1-DAY-ASK-EXTERNAL')
    for ts, px in s.items():
        ts_ns = pd.Timestamp(ts).tz_localize('UTC').value
        p = Price(float(px), 2)
        q = Quantity(100, 0)  # non-zero volume helps bar execution
        yield Bar(last_bt, p, p, p, p, q, ts_ns, ts_ns)
        yield Bar(bid_bt,  p, p, p, p, q, ts_ns, ts_ns)
        yield Bar(ask_bt,  p, p, p, p, q, ts_ns, ts_ns)

# Configure engine and venue
cfg = BacktestEngineConfig()
engine = BacktestEngine(cfg)
cur = Currency('USD', 2, 840, 'US Dollar', CurrencyType.FIAT)
engine.add_venue(
    Venue('SIM'),
    OmsType.NETTING,
    AccountType.CASH,
    [Money(10000000.0, cur)],
    base_currency=cur,
    book_type=BookType.L1_MBP,
    bar_execution=True,
    reject_stop_orders=True,
    support_gtd_orders=True,
    support_contingent_orders=True,
    use_position_ids=True,
    use_random_ids=False,
    use_reduce_only=False,
 )
# Instruments
iids = [to_instrument_id(t) for t in close.columns]
for iid in iids:
    engine.add_instrument(to_equity(iid))
# Strategy
strat_cfg = MomentumConfig(
    instrument_ids=iids,
    lookback_period=LOOKBACK,
    roc_period=ROC,
    num_stocks=NUM_STOCKS,
    rebalance_days=REB_DAYS,
    position_size=POS_SIZE,
    transaction_cost=TX_COST,
    liquidate_on_last_bar=True,
)
strategy = MomentumStrategy(strat_cfg)
engine.add_strategy(strategy)
# Bars
bars = []
for col, iid in zip(close.columns, iids):
    bars.extend(list(bars_from_series(close[col], iid)))
bars.sort(key=lambda b: b.ts_event)
engine.add_data(bars)
engine.run()
result = engine.get_result()
print('Backtest run complete.')


2025-08-22T14:15:58.240766851Z [INFO] BACKTESTER-001.BacktestEngine: =================================================================
2025-08-22T14:15:58.240779625Z [INFO] BACKTESTER-001.BacktestEngine:  NAUTILUS TRADER - Automated Algorithmic Trading Platform
2025-08-22T14:15:58.240781479Z [INFO] BACKTESTER-001.BacktestEngine:  by Nautech Systems Pty Ltd.
2025-08-22T14:15:58.240782591Z [INFO] BACKTESTER-001.BacktestEngine:  Copyright (C) 2015-2024. All rights reserved.
2025-08-22T14:15:58.240785636Z [INFO] BACKTESTER-001.BacktestEngine: =================================================================
2025-08-22T14:15:58.240786478Z [INFO] BACKTESTER-001.BacktestEngine: 
2025-08-22T14:15:58.240787420Z [INFO] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣠⣴⣶⡟⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
2025-08-22T14:15:58.240788532Z [INFO] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣰⣾⣿⣿⣿⠀⢸⣿⣿⣿⣿⣶⣶⣤⣀⠀⠀⠀⠀⠀
2025-08-22T14:15:58.240789724Z [INFO] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⢀⣴⡇⢀⣾⣿⣿⣿⣿⣿⠀⣾⣿⣿⣿⣿⣿⣿⣿⠿⠓⠀⠀⠀⠀
2025-08-22T14:15

In [5]:
# ---------------------------------------------
# Engine-native stats and trade log
# ---------------------------------------------
stats_pnls = getattr(result, 'stats_pnls', {})
stats_returns = getattr(result, 'stats_returns', {})
print('Engine (Nautilus) stats_pnls:')
print(stats_pnls)
print('Engine (Nautilus) stats_returns:')
print(stats_returns)
# Aggregate a single Net PnL if present
net_pnl = None
if isinstance(stats_pnls, dict):
    for k, v in stats_pnls.items():
        if isinstance(v, dict):
            for key in ('pnl_total','total','net','PnL','pnl'):
                if key in v:
                    net_pnl = (net_pnl or 0.0) + float(v[key])
                    break
print('Engine Net PnL (total):', net_pnl if net_pnl is not None else 'N/A')

log_df = pd.DataFrame(getattr(strategy, 'trade_log', []))
if not log_df.empty:
    log_df['ts'] = pd.to_datetime(log_df['ts'], unit='ns', utc=True)
    cols = [c for c in ['side','sym','qty','price','cash','equity_mtm','ts','reason'] if c in log_df.columns]
    log_df = log_df[cols]
    try:
        log_df['equity_mtm_delta'] = log_df['equity_mtm'].diff()
    except Exception:
        pass
    display(log_df)
    print('Symbols traded:', sorted(log_df['sym'].unique().tolist()))
else:
    print('No trades logged.')


Engine (Nautilus) stats_pnls:
{'USD': {'PnL (total)': 15003461.08, 'PnL% (total)': 150.03461079999997, 'Max Winner': 4997282.82, 'Avg Winner': np.float64(1993125.21), 'Min Winner': np.float64(87785.9), 'Min Loser': np.float64(-265540.58), 'Avg Loser': np.float64(-470770.30000000005), 'Max Loser': np.float64(-676000.02), 'Expectancy': np.float64(1500346.108), 'Win Rate': 0.8}}
Engine (Nautilus) stats_returns:
{'Returns Volatility (252 days)': np.float64(0.49454221188365133), 'Average (Return)': np.float64(0.17322781409834434), 'Average Loss (Return)': np.float64(-0.19422918138871992), 'Average Win (Return)': np.float64(0.2650920629701104), 'Sharpe Ratio (252 days)': np.float64(0.9840617606483758), 'Sortino Ratio (252 days)': np.float64(3.184130017438353), 'Profit Factor': np.float64(5.459366323324389), 'Risk Return Ratio': np.float64(0.6866195605878985)}
Engine Net PnL (total): N/A


,side,sym,qty,price,cash,equity_mtm,ts,reason,equity_mtm_delta
0,BUY,267260.KS.SIM,261,36296.84,10000000.00,10000000.00,2023-01-09 00:00:00+00:00,NaN,NaN
1,SELL,267260.KS.SIM,261,40755.20,526522.40,11163629.60,2023-03-08 00:00:00+00:00,NaN,1163629.60
2,BUY,005420.KS.SIM,10,47522.09,526522.40,11163629.60,2023-03-08 00:00:00+00:00,NaN,0.00
3,SELL,005420.KS.SIM,10,56300.68,10688406.34,11251413.14,2023-07-12 00:00:00+00:00,NaN,87783.54
4,BUY,002710.KS.SIM,232,43726.77,10688406.34,11251413.14,2023-07-12 00:00:00+00:00,NaN,0.00
5,SELL,002710.KS.SIM,232,65266.80,1106800.43,16248698.03,2023-10-18 00:00:00+00:00,NaN,4997284.89
6,BUY,022100.KS.SIM,19,54533.96,1106800.43,16248698.03,2023-10-18 00:00:00+00:00,NaN,0.00
7,SELL,022100.KS.SIM,19,40558.14,15212550.72,15983155.38,2024-05-02 00:00:00+00:00,NaN,-265542.65
8,BUY,042700.KS.SIM,107,133931.86,15212550.72,15983155.38,2024-05-02 00:00:00+00:00,NaN,0.00
9,SELL,042700.KS.SIM,107,172524.08,1652445.54,20112522.10,2024-06-13 00:00:00+00:00,NaN,4129366.72


Symbols traded: ['002710.KS.SIM', '003230.KS.SIM', '005420.KS.SIM', '012450.KS.SIM', '022100.KS.SIM', '042700.KS.SIM', '064350.KS.SIM', '267260.KS.SIM', '329180.KS.SIM']


In [6]:
# ---------------------------------------------
# Last 5-day momentum score tables (raw and annualized), transposed
# ---------------------------------------------
pd.set_option('display.max_columns', None)

def compute_score_for_date(close_df, lookback, roc_period, date_idx, annualize=False):
    if date_idx < max(lookback, roc_period):
        return {}
    end = date_idx + 1
    out = {}
    for col in close_df.columns:
        try:
            cur = float(close_df.iloc[date_idx, close_df.columns.get_loc(col)])
            past = float(close_df.iloc[date_idx - roc_period, close_df.columns.get_loc(col)])
            if np.isnan(cur) or np.isnan(past) or past <= 0:
                continue
            roc = (cur - past) / past * 100.0
            roc = max(0.0, min(1.0, float(np.ceil(roc))))
            series = pd.Series(close_df[col].iloc[end - lookback:end]).replace(0, np.nan).dropna()
            if len(series) < 20:
                continue
            x = np.arange(len(series))
            slope = np.polyfit(x, np.log(series.values), 1)[0]
            if annualize:
                slope = (1.0 + slope)**252 - 1.0
            score = roc * slope
            if np.isfinite(score):
                out[col] = float(score)
        except Exception:
            continue
    return out

last_idxs = list(range(max(0, len(close.index) - 5), len(close.index)))
score_cols_raw = {}
score_cols_ann = {}
for i in last_idxs:
    label = str(pd.Timestamp(close.index[i]).date())
    score_cols_raw[label] = pd.Series(compute_score_for_date(close, LOOKBACK, ROC, i, annualize=False))
    score_cols_ann[label] = pd.Series(compute_score_for_date(close, LOOKBACK, ROC, i, annualize=True))
score_table_raw = pd.DataFrame(score_cols_raw)
score_table_ann = pd.DataFrame(score_cols_ann)
if not score_table_raw.empty:
    score_table_raw = score_table_raw.sort_values(by=score_table_raw.columns[-1], ascending=False)
if not score_table_ann.empty:
    score_table_ann = score_table_ann.sort_values(by=score_table_ann.columns[-1], ascending=False)
print('Raw score (ROC × slope), last 5 days:')
display(score_table_raw.T)
print('Annualized score (ROC × ((1+slope)^252 - 1)), last 5 days:')
display(score_table_ann.T)


Raw score (ROC × slope), last 5 days:


,064350.KS,000150.KS,000880.KS,278470.KS,042660.KS,034020.KS,012450.KS,272210.KS,298040.KS,003230.KS,079550.KS,017960.KS,377300.KS,000720.KS,329180.KS,006800.KS,012630.KS,017800.KS,103140.KS,071320.KS,192820.KS,010120.KS,071050.KS,009540.KS,010620.KS,402340.KS,042670.KS,047810.KS,010140.KS,006040.KS,139130.KS,003240.KS,443060.KS,069960.KS,021240.KS,015760.KS,375500.KS,006260.KS,489790.KS,180640.KS,039490.KS,009970.KS,016360.KS,352820.KS,267250.KS,111770.KS,139480.KS,009830.KS,316140.KS,005940.KS,032640.KS,035720.KS,001800.KS,300720.KS,000660.KS,161890.KS,138930.KS,001430.KS,175330.KS,026960.KS,001040.KS,267260.KS,024110.KS,007340.KS,114090.KS,241560.KS,011200.KS,086790.KS,030200.KS,008730.KS,383220.KS,034730.KS,035420.KS,023530.KS,003030.KS,004020.KS,052690.KS,000810.KS,012330.KS,010060.KS,285130.KS,323410.KS,000240.KS,014680.KS,029780.KS,004990.KS,271560.KS,001440.KS,002380.KS,105560.KS,012750.KS,004170.KS,008930.KS,033780.KS,051600.KS,138040.KS,032830.KS,086280.KS,120110.KS,000670.KS,035250.KS,030000.KS,028260.KS,001680.KS,192080.KS,055550.KS,078930.KS,088350.KS,039130.KS,028050.KS,005830.KS,268280.KS,028670.KS,161390.KS,008770.KS,259960.KS,006360.KS,009150.KS,282330.KS,003490.KS,207940.KS,069620.KS,457190.KS,018260.KS,000270.KS,010130.KS,005850.KS,073240.KS,047050.KS,011210.KS,137310.KS,017670.KS,000080.KS,000120.KS,000210.KS,007070.KS,302440.KS,298050.KS,361610.KS,011170.KS,009240.KS,006400.KS,010950.KS,011070.KS,006650.KS,006280.KS,005490.KS,005930.KS,005420.KS,005380.KS,007310.KS,009420.KS,005300.KS,004000.KS,002790.KS,002710.KS,001450.KS,002840.KS,003550.KS,003620.KS,004370.KS,003670.KS,004490.KS,011790.KS,014820.KS,011780.KS,018880.KS,022100.KS,454910.KS,251270.KS,298020.KS,450080.KS,051900.KS,069260.KS,066970.KS,066570.KS,047040.KS,051910.KS,042700.KS,097950.KS,112610.KS,090430.KS,185750.KS,145720.KS,456040.KS,280360.KS,005250.KS,204320.KS,003090.KS,036460.KS,068270.KS,096770.KS,036570.KS,128940.KS,081660.KS,326030.KS,000100.KS,034220.KS,373220.KS,093370.KS
2025-08-13,0.006370,0.006111,0.005771,0.005555,0.005756,0.005610,0.005656,0.005613,0.004860,0.004709,0.004709,0.004383,0.004309,0.004219,0.004202,0.004094,0.003952,0.003648,0.003267,0.003158,0.003146,0.003016,0.003003,0.002972,0.002871,0.002809,0.002692,0.002802,0.002721,0.002691,0.002577,0.002630,0.002561,0.002534,0.002575,0.002481,0.002519,0.002396,0.002436,0.002447,0.002340,0.002396,0.002326,0.002358,0.002160,0.002277,0.002224,0.002031,0.002060,0.002059,0.001942,0.001849,0.001909,0.001875,0.001791,0.001824,0.001749,0.001711,0.001784,0.001858,0.001651,0.001723,0.001678,0.001698,0.001594,0.001689,0.001635,0.001590,0.001657,0.001529,0.001605,0.001433,0.001476,0.001428,0.001501,0.001294,0.001198,0.001278,0.001284,0.001157,0.001191,0.001274,0.001195,0.001030,0.001189,0.001153,0.001222,0.001069,0.001015,0.001125,0.001117,0.001112,0.001069,0.001122,0.001116,0.001181,0.001040,0.000922,0.000946,0.001063,0.000986,0.000899,0.000776,0.000000,0.000776,0.000618,0.000616,0.000585,0.000655,0.000000,0.000553,0.000626,0.000562,0.000525,0.000000,0.000548,0.000000,0.000247,0.000324,0.000323,0.000347,0.000358,0.000353,0.000229,0.000000,0.000464,0.000000,0.000000,0.000000,0.000000,0.000114,0.000096,0.000000,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.000925,-0.0,-0.000000,-0.000000,-0.000000,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.000000,-0.000000,-0.0,-0.0,-0.0,-0.000000,-0.0,-0.0,0.00000,-0.000000,-0.0,-0.0,-0.000000,0.0,-0.0,-0.0,-0.0,-0.0,-0.000035,-0.000000,-0.000117,-0.000000,-0.000000,-0.000285,-0.000426,-0.000000,-0.000000,-0.000508,-0.000502,-0.000769,-0.000935,-0.000000
2025-08-14,0.006389,0.006119,0.005808,0.005626,0.005775,0.005658,0.005663,0.005624,0.004903,0.004735,0.004727,0.004429,0.004352,0.004256,0.004222,0.004118,0.003953,0.003652,0.003309,0.003181,0.003148,0.003052,0.003032,0.003008,0.002896,0.002830,0.002745,0.002814,0.002744,0.002700,0.002602,0.002638,0.002580,0.002549,0.002582,0.002506,0.00252

Annualized score (ROC × ((1+slope)^252 - 1)), last 5 days:


,064350.KS,000150.KS,000880.KS,278470.KS,042660.KS,034020.KS,012450.KS,272210.KS,298040.KS,003230.KS,079550.KS,017960.KS,377300.KS,000720.KS,329180.KS,006800.KS,012630.KS,017800.KS,103140.KS,071320.KS,192820.KS,010120.KS,071050.KS,009540.KS,010620.KS,402340.KS,042670.KS,047810.KS,010140.KS,006040.KS,139130.KS,003240.KS,443060.KS,069960.KS,021240.KS,015760.KS,375500.KS,006260.KS,489790.KS,180640.KS,039490.KS,009970.KS,016360.KS,352820.KS,267250.KS,111770.KS,139480.KS,009830.KS,316140.KS,005940.KS,032640.KS,035720.KS,001800.KS,300720.KS,000660.KS,161890.KS,138930.KS,001430.KS,175330.KS,026960.KS,001040.KS,267260.KS,024110.KS,007340.KS,114090.KS,241560.KS,011200.KS,086790.KS,030200.KS,008730.KS,383220.KS,034730.KS,035420.KS,023530.KS,003030.KS,004020.KS,052690.KS,000810.KS,012330.KS,010060.KS,285130.KS,323410.KS,000240.KS,014680.KS,029780.KS,004990.KS,271560.KS,001440.KS,002380.KS,105560.KS,012750.KS,004170.KS,008930.KS,033780.KS,051600.KS,138040.KS,032830.KS,086280.KS,120110.KS,000670.KS,035250.KS,030000.KS,028260.KS,001680.KS,192080.KS,055550.KS,078930.KS,088350.KS,039130.KS,028050.KS,005830.KS,268280.KS,028670.KS,161390.KS,008770.KS,259960.KS,006360.KS,009150.KS,282330.KS,003490.KS,207940.KS,069620.KS,457190.KS,018260.KS,000270.KS,010130.KS,005850.KS,073240.KS,047050.KS,011210.KS,137310.KS,017670.KS,000080.KS,000120.KS,000210.KS,007070.KS,302440.KS,298050.KS,361610.KS,011170.KS,009240.KS,006400.KS,010950.KS,011070.KS,006650.KS,006280.KS,005490.KS,005930.KS,005420.KS,005380.KS,007310.KS,009420.KS,005300.KS,004000.KS,002790.KS,002710.KS,001450.KS,002840.KS,003550.KS,003620.KS,004370.KS,003670.KS,004490.KS,011790.KS,014820.KS,011780.KS,018880.KS,022100.KS,454910.KS,251270.KS,298020.KS,450080.KS,051900.KS,069260.KS,066970.KS,066570.KS,047040.KS,051910.KS,042700.KS,097950.KS,112610.KS,090430.KS,185750.KS,145720.KS,456040.KS,280360.KS,005250.KS,204320.KS,003090.KS,036460.KS,068270.KS,096770.KS,036570.KS,128940.KS,081660.KS,326030.KS,000100.KS,034220.KS,373220.KS,093370.KS
2025-08-13,3.953937,3.642705,3.263608,3.039334,3.247887,3.094783,3.142427,3.098333,2.392758,2.266651,2.267211,2.010169,1.954824,1.889321,1.876890,1.799705,1.701736,1.503432,1.274856,1.213500,1.206594,1.136001,1.128871,1.112315,1.059246,1.027833,0.968728,1.023941,0.983093,0.968265,0.912833,0.938498,0.905211,0.892285,0.911907,0.866969,0.885333,0.827502,0.846059,0.851212,0.801999,0.827573,0.795968,0.810236,0.722312,0.773965,0.750196,0.667420,0.679620,0.679245,0.630672,0.592993,0.617131,0.603349,0.569837,0.583025,0.553080,0.538653,0.566926,0.596597,0.515401,0.543062,0.525948,0.533590,0.493806,0.529821,0.509534,0.492183,0.517778,0.469784,0.498167,0.434593,0.450177,0.432921,0.459376,0.385279,0.352234,0.379587,0.381663,0.338258,0.349789,0.378246,0.351013,0.296184,0.349042,0.336780,0.360217,0.309050,0.291174,0.327562,0.324836,0.323308,0.308851,0.326458,0.324493,0.346303,0.299451,0.261357,0.269094,0.307052,0.281898,0.254268,0.215768,0.000000,0.215897,0.168335,0.167817,0.158876,0.179302,0.000000,0.149621,0.170874,0.152022,0.141507,0.000000,0.147942,0.000000,0.064226,0.085169,0.084829,0.091444,0.094306,0.092998,0.059431,0.000000,0.123962,0.000000,0.00000,0.000000,0.000000,0.029227,0.024576,0.000000,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.207913,-0.0,-0.000000,-0.00000,-0.000000,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.000000,-0.000000,-0.0,-0.0,-0.0,-0.000000,-0.0,-0.0,0.000000,-0.000000,-0.0,-0.0,-0.000000,0.0,-0.0,-0.0,-0.0,-0.0,-0.008730,-0.000000,-0.029018,-0.000000,-0.000000,-0.069387,-0.101893,-0.000000,-0.000000,-0.120089,-0.118808,-0.176173,-0.210093,-0.000000
2025-08-14,3.977573,3.652032,3.303741,3.111587,3.267681,3.144104,3.149681,3.109126,2.430186,2.288526,2.281657,2.045451,1.987389,1.916276,1.890969,1.816609,1.702719,1.505975,1.299285,1.226091,1.208111,1.155229,1.144444,1.131424,1.072446,1.038574,0.995431,1.030121,0.994701,0.972869,0.924815,0.942202,0.914266,0.899175,0.915153,0.879153,0.887150